In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.fft as fft
import numpy as np
import pandas as pd
import re

In [6]:
device='cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [7]:
from datasets import load_dataset
datasets=load_dataset('wikitext','wikitext-2-raw-v1')

In [9]:
def preprocess_function(sentence):
    text=sentence['text'].lower()
    text=re.sub(r'[^a-zA-Z0-9\s]','',text)
    text=re.sub('\s\s',' ',text)
    sentence['text']=text
    return sentence
datasets['train']=datasets['train'].map(preprocess_function)
datasets['validation']=datasets['validation'].map(preprocess_function)
datasets['test']=datasets['test'].map(preprocess_function)  


datasets['train']=datasets['train'].filter(lambda example: len(example['text'])>20)
datasets['validation']=datasets['validation'].filter(lambda example: len(example['text'])>20)
datasets['test']=datasets['test'].filter(lambda example: len(example['text'])>20)

<>:4: SyntaxWarning: invalid escape sequence '\s'
<>:4: SyntaxWarning: invalid escape sequence '\s'
/var/folders/79/qf4s258d1979pqm6v_z63bmc0000gn/T/ipykernel_1583/1055278940.py:4: SyntaxWarning: invalid escape sequence '\s'
  text=re.sub('\s\s',' ',text)


Map:   0%|          | 0/19530 [00:00<?, ? examples/s]

Map:   0%|          | 0/2083 [00:00<?, ? examples/s]

Map:   0%|          | 0/2383 [00:00<?, ? examples/s]

Filter:   0%|          | 0/19530 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2083 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2383 [00:00<?, ? examples/s]

tokenization

In [10]:
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding 
from transformers import AutoTokenizer

checkpoint='distilbert-base-uncased-finetuned-sst-2-english'
tokenizer=AutoTokenizer.from_pretrained(checkpoint)
def tokenize(sentence):
    sentence=tokenizer(sentence['text'],truncation=True,max_length=512)
    return sentence
tokenized_inputs=datasets.map(tokenize,batched=True)
tokenized_inputs.remove_columns(['text'])
batch=16
data_collator=DataCollatorWithPadding(tokenizer=tokenizer,padding=True,return_tensors='pt')
dataloader=DataLoader(tokenized_inputs['train'],batch_size=batch,shuffle=True,collate_fn=data_collator)

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Map:   0%|          | 0/2333 [00:00<?, ? examples/s]

Map:   0%|          | 0/19067 [00:00<?, ? examples/s]

Map:   0%|          | 0/2034 [00:00<?, ? examples/s]

In [11]:
class PositionalEncodding(torch.nn.Module):
    def __init__(self,d_model,max_sequence_length):
        super().__init__()
        self.d_model=d_model
        self.max_sequence_length=max_sequence_length
        self.positional_encolding=self._generate_positional_encoding().to(device)
    def create_positional_encoding(self):
        positional_encoding=np.zeros((self.max_sequence_length,self.d_model))
        for pos in range(self.max_sequence_length):
            for i in range(0,self.d_model,2):
                positional_encoding[pos,i]=np.sin(pos/(10000**((2*i)/self.d_model)))
                if i+1<self.d_model:
                    positional_encoding[pos,i+1]=np.cos(pos/(10000**((2*(i+1))/self.d_model)))
        return torch.from_numpy(positional_encoding).float()
    def forward(self,x):
        expanded_tensor=torch.unsquedeze(self.positional_encolding,0).expand(x.size(0),-1,-1).to(device)
        return x.to(device)+expanded_tensor[:,:x.size(1),:]
    class PositionalEncodding(nn.Module):
        def __init__(self,sequece_length,vocab_size,embed_dim):
            super(PositionalEncodding,self).__init__()
            self.token_enmbedding=nn.Embedding(vocab_size,embed_dim)
            self.position_embedding=PositionalEncodding(embed_dim,sequence_length)
        def forward(self,x):
            embedded_tokens=self.token_enmbedding(inputs).to(device)
            embedded_positions=self.position_embedding(embedded_tokens).to(device)
            return embedded_positions